In [3]:
import numpy as np
import pandas as pd
import os
import nltk

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
path = "/content/drive/MyDrive/Capstone/Reviews/All"


In [6]:
os.chdir(path)

In [7]:
lib = os.listdir(path)

In [8]:
#!rm -rf "/content/drive/MyDrive/Capstone/Reviews/All/rel_component"

In [9]:
lines = {}
for i in range(len(lib)):
    with open(lib[i], encoding = 'utf-8') as f:
        lines[i] = f.readlines()

In [10]:
a = pd.DataFrame({'reviews' : [], 'sentences' : []})

In [11]:
for i in range(len(lines)):
    for j in range(len(lines[i])):
        d = {'reviews': i, 'sentences': str(i)+ ' ' +lines[i][j]}
        a = a.append(d, ignore_index = True)

In [10]:
#for i in range(len(lines)):
   # for j in range(len(lines[i])):
    #    d = {'reviews': i, 'sentences': i +lines[i][j]}
      #  a = a.append(d, ignore_index = True)

In [11]:
a

,reviews,sentences
0,0.0,0 Display not working properly\n
1,1.0,1 Camera is good battery is good fingure unloc...
2,2.0,2 Nice Phone in samsung budget series I love s...
3,2.0,2 .\n
4,2.0,2 .\n
...,...,...
1023,196.0,196 Not expected from Samsung.\n
1024,196.0,196 Flipkart refused to replace or return.\n
1025,196.0,196 Total waste of money\n
1026,197.0,197 Very nice products\n


In [12]:
pwd

'/content/drive/.shortcut-targets-by-id/1AV_W1oqC95d7JZdOrooG45rpb3yLOKgi/Capstone/Reviews/All'

In [13]:
%cd '/content'

/content


In [14]:
pwd

'/content'

In [15]:
%%capture
!pip install -U spacy-nightly --pre
!pip install -U pip setuptools wheel
!python -m spacy project clone tutorials/rel_component
!python -m spacy download en_core_web_trf
!pip install -U spacy transformers
!pip install typer
!pip install spacy[transformers]

In [16]:
import spacy
import random
import typer
from pathlib import Path
import spacy
from spacy.tokens import DocBin, Doc
from spacy.training.example import Example
from rel_pipe import make_relation_extractor, score_relations
from rel_model import create_relation_model, create_classification_layer, create_instances, create_tensors

In [17]:
text = a['sentences'].to_list()

In [18]:
len(text)

1028

In [19]:
nlp = spacy.load("/content/drive/MyDrive/Capstone/content/model-best")
nlp.add_pipe('sentencizer')

In [20]:
text[:5]

['0 Display not working properly\n',
 '1 Camera is good battery is good fingure unlock sensor is very good and fast it is very thick due to large battery this phone is not for slim phone lovers but phone is very good and price are also good , good job samsung\n',
 '2 Nice Phone in samsung budget series I love samsung products quality , after sale service , the support, security updates all in all great company .\n',
 '2 .\n',
 '2 .\n']

In [21]:
b = pd.DataFrame({'review_file': [], 'sentences':[], 'ASP':[], 'OPI':[] })

In [22]:
b

,review_file,sentences,ASP,OPI


In [23]:
for doc in nlp.pipe(text, disable=["tagger"]):
 
  if doc.ents:
    x = []
    y = []
    k1 = "" #ASP
    k2 = "" #OPI
    
    for e in doc.ents:
      x.append(e.label_)
      y.append(e.text)
   
    for i in range(len(x)):
      k1 = "" #ASP
      k2 = "" #OPI
      
      if (x[i] == 'ASP') & (i!= (len(x)-1)):
        k1 += y[i]

        if (x[i+1] == 'OPI'):
          k2 += y[i+1]

          d1 = {'review_file': doc[0], 'sentences': str(doc[1:]), 'ASP': k1, 'OPI' : k2}
          b = b.append(d1, ignore_index = True)
        else:
          k2 = ""
          d1 = {'review_file': doc[0], 'sentences': str(doc[1:]), 'ASP': k1, 'OPI' : k2}
          b = b.append(d1, ignore_index = True)

      elif (x[i] == 'ASP') & (i == (len(x)-1)):
        k1 += y[i]
        k2 = ""
        d1 = {'review_file': doc[0], 'sentences': str(doc[1:]), 'ASP': k1, 'OPI' : k2}
        b = b.append(d1, ignore_index = True)
        
  
  
  else:
    d1 = {'review_file': doc[0], 'sentences': str(doc[1:])}
    b = b.append(d1, ignore_index = True)

   

/usr/local/lib/python3.7/dist-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In [24]:
b

,review_file,sentences,ASP,OPI
0,0,Display not working properly\n,Display,not working
1,1,Camera is good battery is good fingure unlock ...,Camera,good
2,1,Camera is good battery is good fingure unlock ...,battery,good
3,1,Camera is good battery is good fingure unlock ...,fingure unlock,
4,1,Camera is good battery is good fingure unlock ...,sensor is,very good
...,...,...,...,...
903,194,Performance is also good Display is average al...,Display,
904,195,All are good bt front camera is worst.\n,camera,worst
905,196,"Camera performance & display both are bad, the...",Camera,
906,196,"Camera performance & display both are bad, the...",display,bad


In [25]:
b = b[b["ASP"].notnull()]
#df = b[b["OPI"] != ""]

In [26]:
df = b[b["OPI"] != ""]

In [27]:
df

,review_file,sentences,ASP,OPI
0,0,Display not working properly\n,Display,not working
1,1,Camera is good battery is good fingure unlock ...,Camera,good
2,1,Camera is good battery is good fingure unlock ...,battery,good
4,1,Camera is good battery is good fingure unlock ...,sensor is,very good
5,1,Camera is good battery is good fingure unlock ...,battery,slim
...,...,...,...,...
892,188,Camera is impressive.\n,Camera,impressive
897,193,Battery got drained quickly.\n,Battery,got drained
900,194,Phone is very good but camera of front is not ...,camera,not good
904,195,All are good bt front camera is worst.\n,camera,worst


for doc in nlp.pipe(text, disable=["tagger"]):

  if doc.ents:
    
    k1 = ""  #ASP
    k2 = ""  #OPI

    for e in doc.ents:

      if (e.label_ == 'OPI'):
        e2 = e.text
        if k2 != "":
          k2+= ' , '+ e2
        else:
          k2 += e2  


      elif  (e.label_ == 'ASP'):
        e1 = e.text
        if k1 != "":
          k1+= ' , '+ e1
        else:
          k1 += e1   

    d1 = {'sentences': str(doc) , 'ASP':k1, 'OPI': k2}
    b = b.append(d1, ignore_index = True)

  else:

    d1 = {'sentences': str(doc)}
    b = b.append(d1, ignore_index = True)


In [28]:
df.head(10)

,review_file,sentences,ASP,OPI
0,0,Display not working properly\n,Display,not working
1,1,Camera is good battery is good fingure unlock ...,Camera,good
2,1,Camera is good battery is good fingure unlock ...,battery,good
4,1,Camera is good battery is good fingure unlock ...,sensor is,very good
5,1,Camera is good battery is good fingure unlock ...,battery,slim
7,2,Nice Phone in samsung budget series I love sam...,security updates,great
14,4,"Battery backup , camera and storage all are fa...",storage,fantastic
19,6,This mobile f12 is absolute value for money wi...,battery,thick
20,6,This mobile f12 is absolute value for money wi...,display,much better
21,6,This mobile f12 is absolute value for money wi...,front camera,not good


In [29]:
!pip install vaderSentiment

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 14.6 MB/s eta 0:00:00


In [57]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [58]:
analyzer = SentimentIntensityAnalyzer()

In [59]:
#Create a new column in dataframe with the sentiment analysis of the OPI column
df["OPI_Sentiment"] = df["OPI"].apply(lambda x : analyzer.polarity_scores(x)['compound'] if x != "" else np.nan)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [60]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 271 entries, 0 to 906
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   review_file         271 non-null    object 
 1   sentences           271 non-null    object 
 2   ASP                 271 non-null    object 
 3   OPI                 271 non-null    object 
 4   OPI_Sentiment       271 non-null    float64
 5   Sentence_Sentiment  271 non-null    float64
dtypes: float64(2), object(4)
memory usage: 22.9+ KB


In [34]:
type(df["OPI_Sentiment"][0])

numpy.float64

In [36]:
analyzer = SentimentIntensityAnalyzer()

In [37]:
sentiment_dict = analyzer.polarity_scores("Display not working properly\n")

In [38]:
sentiment_dict 

{'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos': 0.0}

In [61]:
df

,review_file,sentences,ASP,OPI,OPI_Sentiment,Sentence_Sentiment
0,0,Display not working properly\n,Display,not working,0.0000,0.0000
1,1,Camera is good battery is good fingure unlock ...,Camera,good,0.4404,0.9595
2,1,Camera is good battery is good fingure unlock ...,battery,good,0.4404,0.9595
4,1,Camera is good battery is good fingure unlock ...,sensor is,very good,0.4927,0.9595
5,1,Camera is good battery is good fingure unlock ...,battery,slim,0.0000,0.9595
...,...,...,...,...,...,...
892,188,Camera is impressive.\n,Camera,impressive,0.5106,0.5106
897,193,Battery got drained quickly.\n,Battery,got drained,-0.3612,-0.3612
900,194,Phone is very good but camera of front is not ...,camera,not good,-0.3412,-0.2529
904,195,All are good bt front camera is worst.\n,camera,worst,-0.6249,-0.2960


In [62]:
df["Sentence_Sentiment"] = df["sentences"].apply(lambda x : analyzer.polarity_scores(x)['compound'] if x != "" else np.nan)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [63]:
df.head(25)

,review_file,sentences,ASP,OPI,OPI_Sentiment,Sentence_Sentiment
0,0,Display not working properly\n,Display,not working,0.0000,0.0000
1,1,Camera is good battery is good fingure unlock ...,Camera,good,0.4404,0.9595
2,1,Camera is good battery is good fingure unlock ...,battery,good,0.4404,0.9595
4,1,Camera is good battery is good fingure unlock ...,sensor is,very good,0.4927,0.9595
5,1,Camera is good battery is good fingure unlock ...,battery,slim,0.0000,0.9595
7,2,Nice Phone in samsung budget series I love sam...,security updates,great,0.6249,0.9451
14,4,"Battery backup , camera and storage all are fa...",storage,fantastic,0.5574,0.5574
19,6,This mobile f12 is absolute value for money wi...,battery,thick,0.0000,0.7834
20,6,This mobile f12 is absolute value for money wi...,display,much better,0.4404,0.7834
21,6,This mobile f12 is absolute value for money wi...,front camera,not good,-0.3412,0.7834


In [ ]:
df.to_excel("/content/drive/MyDrive/Capstone/Dataset/ASP_DataSet.xlsx", index = False)

In [16]:
df = pd.read_excel("/content/drive/MyDrive/Capstone/Dataset/ASP_DataSet.xlsx")

In [12]:
sent = pd.read_excel("/content/drive/MyDrive/Capstone/Dataset/sent.xlsx")

In [13]:
sent

,review_file,Average of Sentence_Sentiment
0,0,0.000000
1,1,0.959500
2,10,-0.464000
3,100,0.542300
4,101,-0.122050
...,...,...
109,95,0.484233
110,96,0.501650
111,97,-0.170600
112,99,0.690800


In [14]:
def score (x):
  if x<-0.5 :
    return 1
  elif -0.5<= x < 0 :
    return 2
  elif 0 <= x < 0.3:
     return 3
  elif 0.3 <= x < 0.8:
     return 4
  else:
    return 5


In [17]:
sent["rating"] = sent["Average of Sentence_Sentiment"].apply(score) 

In [18]:
sent

,review_file,Average of Sentence_Sentiment,rating
0,0,0.000000,3
1,1,0.959500,5
2,10,-0.464000,2
3,100,0.542300,4
4,101,-0.122050,2
...,...,...,...
109,95,0.484233,4
110,96,0.501650,4
111,97,-0.170600,2
112,99,0.690800,4


In [19]:
sent["rating_label"] = sent["rating"].apply(lambda x: "good" if x > 3 else "bad")

In [20]:
sent_df = sent[["review_file","rating","rating_label"]]

In [21]:
sent_df

,review_file,rating,rating_label
0,0,3,bad
1,1,5,good
2,10,2,bad
3,100,4,good
4,101,2,bad
...,...,...,...
109,95,4,good
110,96,4,good
111,97,2,bad
112,99,4,good


In [22]:
df.head()

,review_file,sentences,ASP,OPI,OPI_Sentiment,Sentence_Sentiment
0,0,Display not working properly\n,Display,not working,0.0000,0.0000
1,1,Camera is good battery is good fingure unlock ...,Camera,good,0.4404,0.9595
2,1,Camera is good battery is good fingure unlock ...,battery,good,0.4404,0.9595
3,1,Camera is good battery is good fingure unlock ...,sensor is,very good,0.4927,0.9595
4,1,Camera is good battery is good fingure unlock ...,battery,slim,0.0000,0.9595


In [23]:
data = df[["review_file","ASP","OPI_Sentiment" ]]

In [74]:
#data.to_excel("/content/drive/MyDrive/Capstone/Dataset/pivot.xlsx", index = False)

In [24]:
pivot_data = pd.read_excel("/content/drive/MyDrive/Capstone/Dataset/pivot.xlsx")

In [25]:
pivot_data.columns

Index(['review_file', '90Hz display', 'accessibility options',
       'advertisement free', 'Back camera', 'Back panel', 'backup', 'battery',
       'Battery backup', 'Biometric', 'body', 'call clarity',
       'Call connectivity', 'Camera', 'charger', 'charging',
       'Color correction', 'Colors', 'cover', 'Display', 'Dolby Atmos',
       'drop notch', 'Face unlock-', 'finger print', 'Fingerprint sensor',
       'finishing', 'focus technology', 'front camera', 'front cameras',
       'gaming', 'graphics', 'Hardware', 'HEAD PHONE', 'internal memory',
       'lock app', 'low light', 'macro', 'micromax', 'Multitasker,', 'One UI',
       'optimisation is', 'outdoor lighting', 'Performance', 'processor',
       'rear camera', 'rear cameras', 'refresh rate', 'resolution', 'screen',
       'screen color', 'screenshot taking', 'security updates', 'selfie',
       'Selfie camera', 'sensor is', 'sensor which', 'SENSORS',
       'service centre', 'Shutter', 'Software', 'sound', 'speaker', 'sto

In [31]:
sent_df = sent_df[sent_df["review_file"] != "Grand Total"]

In [91]:
pivot_data.head()

,review_file,90Hz display,accessibility options,advertisement free,Back camera,Back panel,backup,battery,Battery backup,Biometric,...,Software,sound,speaker,storage,Tap),Temple Run,TFT but,tinted glass,Ui,vpn
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,NaN,NaN,NaN,NaN,NaN,0.2202,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100,0.5423,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,101,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
sent_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 113 entries, 0 to 112
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   review_file   113 non-null    int64 
 1   rating        113 non-null    int64 
 2   rating_label  113 non-null    object
dtypes: int64(2), object(1)
memory usage: 3.5+ KB


In [32]:
sent_df['review_file'] = sent_df['review_file'].astype("int64")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [37]:
pivot_data = pivot_data.sort_values("review_file")

In [35]:
sent_df = sent_df.sort_values("review_file", ascending = True)

In [38]:
pivot_data

,review_file,90Hz display,accessibility options,advertisement free,Back camera,Back panel,backup,battery,Battery backup,Biometric,...,Software,sound,speaker,storage,Tap),Temple Run,TFT but,tinted glass,Ui,vpn
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,NaN,NaN,NaN,NaN,NaN,0.2202,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.5574,NaN,NaN,NaN,NaN,NaN,NaN
77,6,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36,188,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38,193,NaN,NaN,NaN,NaN,NaN,NaN,-0.3612,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39,194,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40,195,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
pivot_data [["rating", "rating_label"]] = sent_df[["rating", "rating_label"]]

In [40]:
final = pivot_data

In [41]:
final

,review_file,90Hz display,accessibility options,advertisement free,Back camera,Back panel,backup,battery,Battery backup,Biometric,...,speaker,storage,Tap),Temple Run,TFT but,tinted glass,Ui,vpn,rating,rating_label
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,bad
1,1,NaN,NaN,NaN,NaN,NaN,NaN,0.2202,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,good
42,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,good
59,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.5574,NaN,NaN,NaN,NaN,NaN,NaN,4,good
77,6,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,good
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36,188,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,good
38,193,NaN,NaN,NaN,NaN,NaN,NaN,-0.3612,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,bad
39,194,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,bad
40,195,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,bad
